In [71]:
import numpy as np
from scipy import optimize as opt
from collections import namedtuple

from LinApp_FindSS import LinApp_FindSS
from LinApp_Deriv import LinApp_Deriv
from LinApp_Solve import LinApp_Solve

ModuleNotFoundError: No module named 'LinApp_FindSS'

In [2]:
# Use a root finder to optimize over the 3 equations from part 1.
params5 = [2.5, 1.5, .98, .4, .5, .10, 0, .05]
def diff5(varvec, params):
    k, r, w = varvec[0], varvec[1], varvec[2]
    gamma, xi, beta, alpha, a, delta, zbar, tau = params
    eulerdiff = 1 - beta * ((r - delta) * (1 - tau) + 1)
    rdiff = r - alpha * k ** (alpha - 1)
    wdiff = w - (1 - alpha) * k ** alpha
    outvec = np.array([eulerdiff, rdiff, wdiff])
    return outvec

roots = opt.root(diff5, [1, 1, 1], args = params5).x
k_ss = roots[0]
r_ss = roots[1]
w_ss = roots[2]
i_ss = k_ss ** .4
y_ss = k_ss * .1
print(" k_ss =", k_ss, "\n r_ss =", r_ss, "\n w_ss =", w_ss, "\n y_ss =", y_ss, "\n i_ss =", i_ss)


 k_ss = 7.287497950692919 
 r_ss = 0.12148227712137491 
 w_ss = 1.327952768351301 
 y_ss = 0.7287497950692919 
 i_ss = 2.2132546139188345


## Exercise 6

In [3]:
params6 = [2.5, 1.5, .98, .4, .5, .10, 0, .05]

#Define tax function
def T(k, r, w, l, params):
    gamma, xi, beta, alpha, a, delta, zbar, tau = params
    T = tau * ( w*l + (r - delta) * k)
    return T

#print(T(1, 1, 1, 1, params6))

#define consumption function (assume T is zero)
def cons(k, r, w, l, params):
    gamma, xi, beta, alpha, a, delta, zbar, tau = params
    c = (1 - tau) * (w * l + (r - delta) * k) - T(k, r, w, l, params6)
    return c

print(cons(1, 1, 1, 1, params6))

#Define Difference function
def diff6(varvec, params):
    k, r, w, l = varvec[0], varvec[1], varvec[2], varvec[3]
    gamma, xi, beta, alpha, a, delta, zbar, tau = params
    c = cons(k, r, w, l, params6)
    cldiff = w * (1 - tau) * c ** (-gamma) + a * (1 - l) ** (-gamma)
    eulerdiff = c ** (-gamma) - c**(-gamma) * (- beta) * ((r - delta) * (1 - tau) + 1)
    rdiff = r - alpha * k ** (alpha - 1)
    wdiff = w - (1 - alpha) * k ** alpha
    outvec = np.array([cldiff, eulerdiff, rdiff, wdiff])
    return outvec

diff6([.4, .4, .4, .4], params6)
#roots = opt.root(diff6, [1, 1, 1, 1], args = params6).x

1.71


array([ 1.37132122e+01,  7.08716511e+01, -2.93144843e-01, -1.58869059e-02])

### Something is likely broken above!

## Baseline Tax Model

### Exercise 4

This is from Chase's notebook on Github! Had to add a few things.

In [16]:
# Parametrize
params = ["a", "alpha", "beta", "delta", "gamma", "rho", "sigma", "tau", "xi", "zbar"]
BaselineModel = namedtuple("BaselineModel", params)

m = BaselineModel(0.50, 0.40, 0.98, 0.10, 2.50, 0.90, 0.02, 0.05, 1.5, 0.0)

Define Functions:

In [17]:
#Utility/Production Functions
def u(m, c, l):
    uc = (c**(1.0 - m.gamma) - 1.0) / (1.0 - m.gamma)
    ul = m.a * ((1.0 - l)**(1.0 - m.xi) - 1.0) / (1.0 - m.xi)
    return uc + ul

def du_dc(m, c, l):
    return c**(-m.gamma)

def du_dl(m, c, l):
    return - m.a * (1.0 - l)**(-m.xi)

def f(m, k, l, z):
    return k**m.alpha * (l * np.exp(z))**(1.0 - m.alpha)

def df_dk(m, k, l, z):
    return m.alpha * (l/k * np.exp(z))**(1.0 - m.alpha)

def df_dl(m, k, l, z):
    return (1.0 - m.alpha) * np.exp(z)**(1.0 - m.alpha) * (k / l)**m.alpha

#Characteristic Equations
def Definitions(m, ktp1, kt, lt, zt):
    wt = df_dl(m, kt, lt, zt)
    rt = df_dk(m, kt, lt, zt)
    Tt = m.tau * (wt*lt + (rt - m.delta)*kt)
    ct = (1 - m.tau) * (wt*lt + (rt - m.delta)*kt) + kt + Tt - ktp1

    ct = max(ct, 1e-8)

    return ct, rt, Tt, wt


def CharacteristicEquations(m, ktp2, ltp1, ktp1, lt, kt, ltm1, ztp1, zt):
    ct, rt, Tt, wt = Definitions(m, ktp1, kt, lt, zt)
    ctp1, rtp1, Ttp1, wtp1 = Definitions(m, ktp2, ktp1, ltp1, ztp1)
    rr_tp1 = (rtp1 - m.delta)*(1 - m.tau) + 1.0

    intertemporal_ee = du_dc(m, ct, lt) - m.beta*du_dc(m, ctp1, ltp1)*rr_tp1
    intratemporal_ee = du_dl(m, ct, lt) + du_dc(m, ct, lt)*wt*(1.0 - m.tau)

    return np.array([intertemporal_ee, intratemporal_ee])

### Exercise 4

In [18]:
# Solve the Steady-state
def solve_ss(m):
    solve_me = lambda x: CharacteristicEquations(m, x[0], x[1], x[0], x[1], x[0], x[1], m.zbar, m.zbar)
    sol = opt.root(solve_me, np.array([0.5, 0.85]))
    return sol.x

kbar, lbar = solve_ss(m)
cbar, rbar, Tbar, wbar = Definitions(m, kbar, kbar, lbar, m.zbar)

 cbar =  0.8607032061550788 
 rbar =  0.12148227712083877 
 Tbar =  0.04303516030775391 
 wbar = 1.3279527683552124


### Exercises 4/5

In [63]:
'''
dy/dx will be returned as a matrix of derivatives with the columns 
being one element of x, and the rows being one element of y. (basically
a derivative matrix) The xs are in the order as defined in the namedtuple.
The ys are in the same order as they are in the problem, see the output
of "get_outcomes"
'''

def get_outcomes(m):
    '''
    This is the function I'll be differentiating. It takes the parameters
    m as inputs, and returns the important outputs as a list in the 
    appropriate order.
    '''
    kbar, lbar = solve_ss(m)
    cbar, rbar, Tbar, wbar = Definitions(m, kbar, kbar, lbar, m.zbar)
    ybar = f(m, kbar, lbar, m.zbar)
    ibar = ybar - cbar
    return kbar, cbar, rbar, wbar, lbar, Tbar, ybar, ibar

def Dmat5(x, y, f, dist = 10e-6):
    m = len(y)
    n = len(x)
    #Initialize Jacobian
    Dmat = np.zeros((m, n))
    for i in range(m):
        for j in range(n):
            xnew = np.array(x)
            xnew[j] = xnew[j] - 10e-6
            xnew = BaselineModel(*xnew)
            Dmat[i, j] = (f(x)[i] - f(xnew)[i]) / dist
    return Dmat

ss = get_outcomes(m)
Derivs = Dmat5(m, ss, get_outcomes)
print(" Steady state values: \n", ss, "\n Derivatives: \n", Derivs)
print("Examples: \n Derivative of interest rate w.r.t beta: \n", Derivs[2, 2], \
      "\n Derivative of output w.r.t zbar: \n", Derivs[6, 9], \
      "\n Derivative of capital stock w.r.t. alpha: \n", Derivs[0, 1], \
      "\n Derivative of consumption w.r.t. gamma: \n", Derivs[1, 4], \
      "\n Derivative of total tax w.r.t. tau \n", Derivs[5, 7])

print("That consumption number looks... weird... but everything else seems like it's going the correct direction!")

 Steady state values: 
 (4.225229026815977, 0.8607032061550788, 0.12148227712083877, 1.3279527683552124, 0.5797914531671479, 0.04303516030775391, 1.2832261088366759, 0.4225229026815971) 
 Derivatives: 
 [[-1.84927301e+00  2.59839699e+01  6.54224134e+01 -4.83544084e+01
   1.38699492e-01  0.00000000e+00  0.00000000e+00 -2.32321275e+00
  -8.01657851e-01  2.83828090e+00]
 [-3.76707418e-01  2.08513434e+00  1.75110016e+00 -3.51149925e+00
   2.82538834e-02  0.00000000e+00  0.00000000e+00 -2.34390902e-01
  -1.63302315e-01  5.78173980e-01]
 [ 1.99562589e-09 -7.03187508e-09 -1.09604572e+00  1.00000001e+00
   6.89726054e-10  0.00000000e+00  0.00000000e+00  2.26126864e-02
  -4.42562653e-09  2.48412402e-10]
 [-1.45883305e-08  4.39568631e+00  7.98683048e+00 -7.28799800e+00
  -5.04041253e-09  0.00000000e+00  0.00000000e+00 -1.64790161e-01
   3.22408766e-08  1.32794613e+00]
 [-2.53759645e-01 -7.69361431e-01  2.60046612e-01  1.31984080e+00
   1.90325311e-02  0.00000000e+00  0.00000000e+00 -1.38922976e-

## Exercise 6

In [70]:
def linearize(m, kbar, lbar, eps=1e-6):
    # Compute ss gamma
    x = np.array([kbar, lbar, kbar, lbar, kbar, lbar, 0.0, 0.0])
    Gamma_ss = CharacteristicEquations(m, kbar, lbar, kbar, lbar, kbar, lbar, 0.0, 0.0)

    # Allocate space for derivative matrix
    neq = len(Gamma_ss)
    Dmat = np.empty((neq, 8))

    CEsplat = lambda x: CharacteristicEquations(m, *list(x))
    for i in range(8):
        xtilde = np.array([kbar, lbar, kbar, lbar, kbar, lbar, 0.0, 0.0])
        xtilde[i] += eps
        Dmat[:, i] = derivative(CEsplat, x, xtilde)

    F = Dmat[:, :2]
    G = Dmat[:, 2:4]
    H = Dmat[:, 4:6]
    L = Dmat[:, 6]
    M = Dmat[:, 7]
    #N is 1x1 so just a constant
    N = m.rho
    
    return F, G, H, L, M, N

F, G, H, L, M, N = linearize(m, kbar, lbar)
F, G, H, L, M, N


(array([[ 4.22623769e-06, -5.44191867e-06],
        [ 0.00000000e+00,  0.00000000e+00]]),
 array([[-8.56661520e-06,  5.61221554e-06],
        [-5.33163184e-06,  1.48988829e-05]]),
 array([[4.31700912e-06, 0.00000000e+00],
        [5.27237248e-06, 0.00000000e+00]]),
 array([-3.1551835e-06,  0.0000000e+00]),
 array([3.25392039e-06, 3.00365571e-06]),
 0.9)